In [1]:
%%capture
!pip install datasets
!pip install transformers
!pip install torchaudio
!pip install librosa
!pip install jiwer
!pip install pandas
!pip install scikit-learn
!pip install numpy

In [2]:
from transformers import Wav2Vec2CTCTokenizer

# Create a Wav2Vec2CTCTokenizer object with the generated vocabulary json with the required tokens
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [3]:
from transformers import Wav2Vec2FeatureExtractor

# Create a Wav2Vec2FeatureExtractor object for the audio
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True,
                                             return_attention_mask=True)


In [4]:
from transformers import Wav2Vec2Processor
# Create Wav2Vec2Processor object using the created feature_extractor and tokenizer
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [5]:
from datasets import load_from_disk
dataset_train = load_from_disk("./processed_data/train")

In [6]:
dataset_test = load_from_disk("./processed_data/test")

In [13]:
import librosa
import numpy as np

In [14]:
def resample(batch):
    batch["speech"] = librosa.resample(np.asarray(batch["speech"]), 48_000, 16_000)
    batch["sampling_rate"] = 16_000
    return batch

In [15]:
dataset_train = dataset_train.map(resample)

In [16]:
dataset_test = dataset_test.map(resample)

In [19]:
from transformers import Wav2Vec2Processor
# Create Wav2Vec2Processor object using the created feature_extractor and tokenizer
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)
def prepare_dataset(batch):
    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [21]:
dataset_train = dataset_train.map(prepare_dataset, remove_columns=dataset_train.column_names, batch_size=8,batched=True)

d:\course_tbc\4th year\pp\fyp\project\venv\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


OSError: [Errno 28] Error writing bytes to file. Detail: [errno 28] No space left on device